In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from dotenv import load_dotenv

print(pd.__version__)
warnings.filterwarnings('ignore')
load_dotenv()
sns.set_style()

In [ ]:
data = pd.read_csv('enrollment_code_introspect.csv', encoding='iso-8859-1')
data.head()



In [ ]:
data.shape

In [ ]:
data.columns

## Remove test data organization

In [ ]:
data['organization_name'].value_counts()

In [ ]:
data = data[data['organization_name'] != "Â¡TEST! Data Organization"]
data['organization_name'].value_counts()

## Merge protocols into single column

In [ ]:
def create_protocol_column(row):
    protocols = []
    fields = ['is_ftm', 'is_cfa', 'is_carnmpp', 'is_comet', 'is_nerp']
    for field in fields:
        if row[field]:
            protocols.append(field)
    return ', '.join(protocols)

data['protocol'] = data.apply(create_protocol_column, axis=1)

data.describe(exclude='number').T

protocol_counts = data['protocol'].value_counts()
protocol_counts

## Drop cruft columns

In [ ]:
null_columns = data.columns[data.isnull().all()]
print(null_columns)


In [ ]:
columns_to_remove = ['is_ftm', 'is_cfa', 'is_carnmpp', 'is_comet', 'is_nerp', 'region_boundary', 'parent_region_boundary', 'parent_organization_name', 'campaign_kind']
data.drop(columns=columns_to_remove + list(null_columns), inplace=True)

data.head()

## Data info

In [ ]:
data.describe(exclude='number').T.assign(non_null_counts=data.count())

## Regions

In [ ]:
pd.set_option('display.max_rows', None)
region_counts = data['region_name'].value_counts(dropna=False)
print(region_counts)


### Parent regions (most don't have parent regions)

In [ ]:
parent_region_counts = data['parent_region_name'].value_counts()

for parent_region, count in parent_region_counts.items():
    if count > 8:
        print(f"{parent_region}: {count}")
        region_counts = data[data['parent_region_name'] == parent_region]['region_name'].value_counts()
        for region, region_count in region_counts.items():
            print(f"    {region}: {region_count}")
    else:
        print(f"{parent_region}: {count}")
            



In [ ]:
# Sort rows by 'protocol' and then by 'campaign_name'. Sort in place and return all columns.
data.sort_values(by=['protocol', 'campaign_name'], inplace=True)
# Reset the index
data.reset_index(drop=True, inplace=True)

# Move 'protocol' column to the first position and 'campaign_name' to the second position
cols = ['protocol', 'campaign_name'] + [col for col in data.columns if col not in ['protocol', 'campaign_name']]
data = data[cols]
data.to_csv('cleaned_data.csv')


